# ОПИСАНИЕ ЗАДАЧИ

Данные:
- чековые данные (transactions.parquet, для чтения через pandas дополнительно нужно установить библиотеку pyarrow)
- справочник товаров (materials.csv)
- справочник магазинов (plants.csv)
- справочник клиентов (clients.csv)
Более подробное описание данных дано в файле Data Description.

Цель: 
1) проанализировать данные и определить оптимальную методологию определения отточных клиентов
2) разработать модель вероятности оттока клиентов по выбранной вами методологии
3) дать интерпретацию разработанной модели, ответить на вопросы: какие признаки наиболее влияют на отток клиентов


# ЧАСТЬ 1. ПРЕДОБРАБОТКА ВХОДНЫХ ДАННЫХ

### Загрузка библиотек

In [1]:
import pyarrow.parquet as pq
import pandas as pd
from pathlib import Path
import numpy as np
import pickle

# 1. Загрузка данных

In [2]:
workdir = str(Path().absolute())

In [3]:
transactions = pq.read_table(workdir+'/transactions.parquet')
transactions = transactions.to_pandas()
clients = pd.read_csv(workdir+'/clients.csv')
materials = pd.read_csv(workdir+'/materials.csv')
plants = pd.read_csv(workdir+'/plants.csv')

Посмотрим на данные.

In [4]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32109414 entries, 0 to 32109413
Data columns (total 9 columns):
chq_id          object
plant           object
chq_date        datetime64[ns]
chq_position    object
client_id       object
material        object
sales_count     float64
sales_sum       float64
is_promo        int64
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 2.2+ GB


In [5]:
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99995 entries, 0 to 99994
Data columns (total 4 columns):
client_id    99995 non-null object
gender       99640 non-null object
city         99995 non-null object
birthyear    98219 non-null float64
dtypes: float64(1), object(3)
memory usage: 3.1+ MB


In [6]:
materials.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105609 entries, 0 to 105608
Data columns (total 8 columns):
material            105609 non-null object
hier_level_1        105609 non-null object
hier_level_2        105609 non-null object
hier_level_3        105609 non-null object
hier_level_4        105609 non-null object
vendor              105609 non-null object
is_private_label    105609 non-null int64
is_alco             105609 non-null int64
dtypes: int64(2), object(6)
memory usage: 6.4+ MB


In [7]:
plants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 388 entries, 0 to 387
Data columns (total 3 columns):
plant         388 non-null object
plant_type    388 non-null object
city          388 non-null object
dtypes: object(3)
memory usage: 9.2+ KB


In [8]:
print(80*'*')
print('transactions:')
display(transactions.head())

print(80*'*')
print('clients:')
display(clients.head())

print(80*'*')
print('materials:')
display(materials.head())

print(80*'*')
print('plants:')
display(plants.head())

********************************************************************************
transactions:


,chq_id,plant,chq_date,chq_position,client_id,material,sales_count,sales_sum,is_promo
0,ce5b3d1c531348c5d0d8e859b6d0cf91,7cd86ecb09aa48c6e620b340f6a74592,2016-11-01,004,90fca68123e692a005a4edeadd94b2f2,232cfe8b28970434f685b737737b26d4,2.0,146.98,0
1,ce5b3d1c531348c5d0d8e859b6d0cf91,7cd86ecb09aa48c6e620b340f6a74592,2016-11-01,003,90fca68123e692a005a4edeadd94b2f2,12ec64451b8e426db16f3f05ea770901,1.0,249.99,1
2,ce5b3d1c531348c5d0d8e859b6d0cf91,7cd86ecb09aa48c6e620b340f6a74592,2016-11-01,012,90fca68123e692a005a4edeadd94b2f2,1212ff8ecf53aebef093ca8ae216db68,1.0,47.99,1
3,ce5b3d1c531348c5d0d8e859b6d0cf91,7cd86ecb09aa48c6e620b340f6a74592,2016-11-01,013,90fca68123e692a005a4edeadd94b2f2,1212ff8ecf53aebef093ca8ae216db68,1.0,47.99,1
4,ce5b3d1c531348c5d0d8e859b6d0cf91,7cd86ecb09aa48c6e620b340f6a74592,2016-11-01,011,90fca68123e692a005a4edeadd94b2f2,571c6d3b559db8445e05265649eb30ab,2.0,53.98,1


********************************************************************************
clients:


,client_id,gender,city,birthyear
0,1a47d62dddacc03fe90c15652f7ae1a4,M,Other,1990.0
1,9cb909f701d25d548e953bff81192b56,F,Other,1969.0
2,d5da0f5b841b4f83383202807995027a,M,Other,1976.0
3,13ed7f16810b17b8cee6de834ac79a48,F,Moscow,1966.0
4,215fe3ea7d5bf0415e5504e2a7f33551,F,Other,1988.0


********************************************************************************
materials:


,material,hier_level_1,hier_level_2,hier_level_3,hier_level_4,vendor,is_private_label,is_alco
0,35cbdf61de9e19c8b417327aaef14c88,NONFOOD,ea5d2f1c4608232e07d3aa3d998e5135,99cad265a1768cc2dd013f0e740300ae,9eed45f71360b4b1e2590637467220e5,212a38db0ddcd009f1e164cc8483485c,0,0
1,c0b0bf24d4ec71da3d304f761ec555d8,NONFOOD,ea5d2f1c4608232e07d3aa3d998e5135,99cad265a1768cc2dd013f0e740300ae,de2eb747e0896c050905a6b635ab800a,9b0b6c7d55413ad3b67761b7b125b534,0,0
2,f0fc5e654a81a7c4b8ba8d7c26546e14,NONFOOD,ea5d2f1c4608232e07d3aa3d998e5135,99cad265a1768cc2dd013f0e740300ae,8a969031832c535daf96e0c2aed8e814,9b0b6c7d55413ad3b67761b7b125b534,0,0
3,353693e64fb5f9e2d29746d7fe6edf1e,NONFOOD,ea5d2f1c4608232e07d3aa3d998e5135,b58f7d184743106a8a66028b7a28937c,4c0dc012ebb679a18b244c53c6f59b5a,a3c8be149d718771e892619bd310b961,0,0
4,5d9e0b4302ce95448cae72165ff4cf5b,NONFOOD,ea5d2f1c4608232e07d3aa3d998e5135,b58f7d184743106a8a66028b7a28937c,4c0dc012ebb679a18b244c53c6f59b5a,a3c8be149d718771e892619bd310b961,0,0


********************************************************************************
plants:


,plant,plant_type,city
0,95b09698fda1f64af16708ffb859eab9,HM,St. Petersburg
1,926abae84a4bd33c834bc6b981b8cf30,HM,St. Petersburg
2,ae2bac2e4b4da805d01b2952d7e35ba4,HM,St. Petersburg
3,0e7e3cf0ded4d9db8b376b317c007f99,HM,St. Petersburg
4,540bd55a2cf295b8ea9cd78650e89d03,HM,St. Petersburg


Выводы:
* данные transactions занимают очень много памяти
* для некоторых данных возможно уменьшение размера потребляемой памяти
* данные clients содержат пропущенные значения в столбцах gender и birthyear

# 2. Предобработка данных

### Уменьшение размера уникальных ключей

Посмотрим на количество уникальных ключей в каждом датафрейме.

In [9]:
print('transactions:')
print(f'     Уникальных chq_id: {transactions.chq_id.nunique()}')
print(f'     Уникальных client_id: {transactions.client_id.nunique()}')
print(f'     Уникальных material: {transactions.material.nunique()}')
print(f'     Уникальных plant: {transactions.plant.nunique()}')
print('clients:')
print(f'     Уникальных client_id: {clients.client_id.nunique()}')
print('materials:')
print(f'     Уникальных material: {materials.material.nunique()}')
print('plants:')
print(f'     Уникальных plant: {plants.plant.nunique()}')

transactions:
     Уникальных chq_id: 2964775
     Уникальных client_id: 100000
     Уникальных material: 105694
     Уникальных plant: 388
clients:
     Уникальных client_id: 99995
materials:
     Уникальных material: 105609
plants:
     Уникальных plant: 388


Уникальных ключей намного меньше, чем позволяет хранить их длинный шестнадцатиричный формат. В рамках данного анализа целесообразно перевести эти ключи в числовой формат меньшей размерности. Это позволит существенно уменьшить потребление памяти.

In [10]:
client_ids = pd.DataFrame({'client_id':transactions.client_id.drop_duplicates()\
                                                       .append(clients.client_id.drop_duplicates())\
                                                       .drop_duplicates()})
client_ids['num'] = client_ids['client_id'].astype('category').cat.codes

material_ids = pd.DataFrame({'material':transactions.material.drop_duplicates()\
                                                       .append(materials.material.drop_duplicates())\
                                                       .drop_duplicates()})
material_ids['num'] = material_ids['material'].astype('category').cat.codes

plant_ids = pd.DataFrame({'plant':transactions.plant.drop_duplicates()\
                                                       .append(plants.plant.drop_duplicates())\
                                                       .drop_duplicates()})
plant_ids['num'] = plant_ids['plant'].astype('category').cat.codes

chq_ids = pd.DataFrame({'chq_id':transactions.chq_id.drop_duplicates()})
chq_ids['num'] = chq_ids['chq_id'].astype('category').cat.codes

In [11]:
transactions = transactions.merge(client_ids, how='left', on='client_id')\
                           .drop(columns='client_id')\
                           .rename(columns={'num':'client_id'})
transactions = transactions.merge(material_ids, how='left', on='material')\
                           .drop(columns='material')\
                           .rename(columns={'num':'material'})
transactions = transactions.merge(plant_ids, how='left', on='plant')\
                           .drop(columns='plant')\
                           .rename(columns={'num':'plant'})
transactions = transactions.merge(chq_ids, how='left', on='chq_id')\
                           .drop(columns='chq_id')\
                           .rename(columns={'num':'chq_id'})
clients = clients.merge(client_ids, how='left', on='client_id')\
                           .drop(columns='client_id')\
                           .rename(columns={'num':'client_id'})
materials = materials.merge(material_ids, how='left', on='material')\
                           .drop(columns='material')\
                           .rename(columns={'num':'material'})
plants = plants.merge(plant_ids, how='left', on='plant')\
                           .drop(columns='plant')\
                           .rename(columns={'num':'plant'})

### Предобработка transactions

Посмотрим на object-данные по позиции в чеке (chq_position).

In [12]:
transactions['chq_position'].unique()

array(['004', '003', '012', '013', '011', '001', '006', '010', '007',
       '005', '008', '002', '009', '022', '025', '023', '049', '030',
       '046', '038', '041', '024', '045', '018', '033', '034', '019',
       '042', '020', '021', '035', '016', '029', '015', '047', '026',
       '031', '043', '014', '039', '036', '027', '017', '032', '040',
       '048', '044', '028', '037', '068', '064', '073', '055', '052',
       '058', '065', '074', '075', '077', '054', '072', '066', '059',
       '060', '067', '051', '078', '069', '056', '081', '076', '080',
       '071', '061', '082', '050', '062', '070', '079', '053', '063',
       '057', '085', '088', '095', '087', '089', '096', '103', '086',
       '100', '097', '092', '098', '105', '084', '093', '091', '101',
       '102', '083', '104', '099', '094', '090', '128', '129', '133',
       '126', '146', '136', '137', '124', '116', '113', '141', '131',
       '106', '148', '114', '117', '144', '119', '120', '122', '112',
       '134', '145',

Все данные являются числовыми. Убедимся, что данные формата "00000000XX" эквиваленты данным формата "0XX".

In [13]:
transactions[transactions.chq_position=='0000000010'].head()

,chq_date,chq_position,sales_count,sales_sum,is_promo,client_id,material,plant,chq_id
10887678,2017-09-05,0000000010,1.00,369.39,0,46747,92735,49,106596
10887692,2017-09-05,0000000010,2.07,98.10,0,82757,40490,49,1842934
10887707,2017-09-05,0000000010,1.00,68.89,1,14700,91883,49,1514007
10887729,2017-09-05,0000000010,1.00,39.99,0,95,26915,49,2507697
10887740,2017-09-05,0000000010,1.00,114.99,1,99480,5407,49,1939753


In [14]:
transactions[transactions['chq_id'] == 106596].sort_values(by='chq_position')

,chq_date,chq_position,sales_count,sales_sum,is_promo,client_id,material,plant,chq_id
10887667,2017-09-05,0000000001,1.000,178.99,0,46747,59588,49,106596
10887673,2017-09-05,0000000002,1.000,182.99,0,46747,82712,49,106596
10887662,2017-09-05,0000000003,1.000,89.89,1,46747,7415,49,106596
10887670,2017-09-05,0000000004,1.000,89.89,1,46747,7415,49,106596
10887665,2017-09-05,0000000005,1.000,39.89,1,46747,90001,49,106596
10887674,2017-09-05,0000000006,0.340,16.11,0,46747,40490,49,106596
10887672,2017-09-05,0000000007,1.000,350.99,0,46747,21786,49,106596
10887676,2017-09-05,0000000008,1.000,289.89,0,46747,104758,49,106596
10887679,2017-09-05,0000000009,1.000,368.67,0,46747,92735,49,106596
10887678,2017-09-05,0000000010,1.000,369.39,0,46747,92735,49,106596


Переведем chq_position в формат int.

In [15]:
transactions.chq_position = transactions.chq_position.astype(int)

Посмотрим на данные is_promo.

In [16]:
transactions.is_promo.unique()

array([0, 1], dtype=int64)

Переведем их в boolean-формат.

In [17]:
transactions.is_promo = transactions.is_promo.astype(bool)

Попробуем уменьшить размер типов данных float.

In [18]:
for col in ['sales_count', 'sales_sum']:
    c_min = transactions[col].min()
    c_max = transactions[col].max()
    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
        transactions[col] = transactions[col].astype(np.float16)
    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
        transactions[col] = transactions[col].astype(np.float32)
    else:
        transactions[col] = transactions[col].astype(np.float64)  

In [19]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32109414 entries, 0 to 32109413
Data columns (total 9 columns):
chq_date        datetime64[ns]
chq_position    int32
sales_count     float16
sales_sum       float32
is_promo        bool
client_id       int32
material        int32
plant           int16
chq_id          int32
dtypes: bool(1), datetime64[ns](1), float16(1), float32(1), int16(1), int32(4)
memory usage: 1.2 GB


### Предобработка clients

Посмотрим на данные birthyear.

In [20]:
clients.birthyear.unique()

array([1990., 1969., 1976., 1966., 1988., 1987., 1997., 1948., 1984.,
       1952., 1983., 1972., 1992., 2003., 1993., 1980., 1975., 1967.,
       1963., 1946., 1970., 1991., 1957., 2005., 1982., 1986., 1989.,
       1977., 1998., 1951., 1953., 1961., 1978., 1959., 1973., 1943.,
       1958., 1964., 1954., 1955., 2002., 1994., 1962., 1981., 1949.,
       1974.,   nan, 2001., 1979., 1956., 1960., 1995., 1968., 1934.,
       1933., 1985., 1996., 1965., 1928., 1945., 2000., 1971., 1999.,
       1947., 1944., 1950., 1939., 1937., 1940., 1912., 1936., 2006.,
       1935., 1927., 1930., 1900., 1919., 1931., 1938., 1942., 1941.,
       2011., 2004., 1932., 1917., 1922., 1925., 2049., 1911., 1923.,
       1921., 2007., 2019., 1929., 2009., 2008., 2012., 2015., 1926.,
       1918., 2016., 1915., 1914., 1901., 1920., 2010., 1913., 1909.])

In [21]:
clients.birthyear.min()

1900.0

Временно заменим nan-значения birthyear на 1899 и преобразуем столбец в int-формат.

In [22]:
clients.birthyear.fillna(1899., inplace=True)
clients.birthyear = clients.birthyear.astype(int)

Посмотрим на данные gender.

In [23]:
clients.gender.unique()

array(['M', 'F', nan], dtype=object)

Временно заменим nan-значения gender на 0 и преобразуем столбец в int-формат.

In [24]:
clients.gender.fillna(0, inplace=True)
clients.gender = clients.gender.astype('category').cat.codes

Посмотрим на данные city.

In [25]:
clients.city.unique()

array(['Other', 'Moscow', 'St. Petersburg'], dtype=object)

Преобразуем столбец в категориальный формат. Категории городов также потребуются для преобразования данных plants.

In [26]:
city_ids = pd.DataFrame({'city':clients.city.drop_duplicates()})
city_ids['num'] = city_ids['city'].astype('category').cat.codes
clients = clients.merge(city_ids, how='left', on='city')\
                           .drop(columns='city')\
                           .rename(columns={'num':'city'})

Проверим в transactions наличие данных по всем пользователям:

In [27]:
missing_clients = transactions[~transactions.client_id.isin(clients.client_id)].client_id.unique()
print(f"Клиенты с неизвестными данными:{missing_clients}")
print(f"Количество транзакций неизвестных клиентов:{transactions[transactions.client_id.isin(missing_clients)].client_id.count()}")

Клиенты с неизвестными данными:[83888 24874 42601 94757 57927]
Количество транзакций неизвестных клиентов:1616


Учитывая очень маленькое количество неизвестных клиентов и записей по ним, удалим их из данных.

In [28]:
transactions = transactions[transactions.client_id.isin(clients.client_id)]

Для задачи потребуется информация о стоимости единицы товара. Для этого добавим дополнительный столбец price (стоимость единицы товара).

In [29]:
transactions['price'] = transactions['sales_sum'] / transactions['sales_count']

Результат обработки:

In [30]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32107798 entries, 0 to 32109413
Data columns (total 10 columns):
chq_date        datetime64[ns]
chq_position    int32
sales_count     float16
sales_sum       float32
is_promo        bool
client_id       int32
material        int32
plant           int16
chq_id          int32
price           float32
dtypes: bool(1), datetime64[ns](1), float16(1), float32(2), int16(1), int32(4)
memory usage: 1.3 GB


In [31]:
clients.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99995 entries, 0 to 99994
Data columns (total 4 columns):
gender       99995 non-null int8
birthyear    99995 non-null int32
client_id    99995 non-null int32
city         99995 non-null int8
dtypes: int32(2), int8(2)
memory usage: 1.7 MB


### Предобработка materials

Преобразуем столбцы с подкатегориями товара и производителями в категориальный формат.

In [32]:
materials.hier_level_1 = materials.hier_level_1.astype('category').cat.codes
materials.hier_level_2 = materials.hier_level_2.astype('category').cat.codes
materials.hier_level_3 = materials.hier_level_3.astype('category').cat.codes
materials.hier_level_4 = materials.hier_level_4.astype('category').cat.codes

In [33]:
materials.vendor = materials.vendor.astype('category').cat.codes

Посмотрим на столбцы is_private_label, is_alco.

In [34]:
materials.is_private_label.unique()

array([0, 1], dtype=int64)

In [35]:
materials.is_alco.unique()

array([0, 1], dtype=int64)

Преобразуем их в формат boolean.

In [36]:
materials.is_private_label = materials.is_private_label.astype(bool)
materials.is_alco = materials.is_alco.astype(bool)

Проверим в transactions наличие данных по всем товарам:

In [37]:
missing_materials = transactions[~transactions.material.isin(materials.material)].material.unique()
print(f"Товары с неизвестными данными:{missing_materials}")
print(f"Количество транзакций с неизвестными товарами:{transactions[transactions.material.isin(missing_materials)].material.count()}")

Товары с неизвестными данными:[100875  39589  82163  71944  18615  71192  15655  53633 105683  71075
  28318  40196  95433 105507  18963  86149  80837  79434  16381  82082
  49961  93521  99073  35363  59542  60861  46026  66605    640  31925
  96137  75809  41999  83754  99317  86393   6929  12433 104709  88527
  53070  92119  71725  41336 104226  29569  41492   7622 101893  65099
  83711  30717  41811  89172  11258  41434  19921  97082  96519 104774
  39447  37517  36710  21241  55791  48164  56336  24778  94728  88477
  11094  18273  18930  64782  75889  21926  74885  91168  83846   6926
  79991  26307  77911 100613  50054]
Количество транзакций с неизвестными товарами:13139


Учитывая очень маленькое количество неизвестных товаров и записей по ним, удалим их из данных.

In [38]:
transactions = transactions[transactions.material.isin(materials.material)]

Результат обработки:

In [39]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32094659 entries, 0 to 32109413
Data columns (total 10 columns):
chq_date        datetime64[ns]
chq_position    int32
sales_count     float16
sales_sum       float32
is_promo        bool
client_id       int32
material        int32
plant           int16
chq_id          int32
price           float32
dtypes: bool(1), datetime64[ns](1), float16(1), float32(2), int16(1), int32(4)
memory usage: 1.3 GB


In [40]:
materials.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105609 entries, 0 to 105608
Data columns (total 8 columns):
hier_level_1        105609 non-null int8
hier_level_2        105609 non-null int8
hier_level_3        105609 non-null int16
hier_level_4        105609 non-null int16
vendor              105609 non-null int16
is_private_label    105609 non-null bool
is_alco             105609 non-null bool
material            105609 non-null int32
dtypes: bool(2), int16(3), int32(1), int8(2)
memory usage: 2.2 MB


### Предобработка plants

Посмотрим на данные plant_type.

In [41]:
plants.plant_type.unique()

array(['HM', 'SM'], dtype=object)

Преобразуем их в категориальный формат

In [42]:
plants.plant_type = plants.plant_type.astype('category').cat.codes

Посмотрим на данные city.

In [43]:
plants.city.unique()

array(['St. Petersburg', 'Other', 'Moscow'], dtype=object)

Преобразуем их аналогично clients.

In [44]:
plants = plants.merge(city_ids, how='left', on='city')\
                           .drop(columns='city')\
                           .rename(columns={'num':'city'})

Проверим в transactions наличие данных по всем магазинам:

In [45]:
missing_plants = transactions[~transactions.plant.isin(plants.plant)].plant.unique()
print(f"Магазины с неизвестными данными:{missing_plants}")
print(f"Количество транзакций в неизвестных магазинах:{transactions[transactions.plant.isin(missing_plants)].plant.count()}")

Магазины с неизвестными данными:[]
Количество транзакций в неизвестных магазинах:0


Результат обработки:

In [46]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32094659 entries, 0 to 32109413
Data columns (total 10 columns):
chq_date        datetime64[ns]
chq_position    int32
sales_count     float16
sales_sum       float32
is_promo        bool
client_id       int32
material        int32
plant           int16
chq_id          int32
price           float32
dtypes: bool(1), datetime64[ns](1), float16(1), float32(2), int16(1), int32(4)
memory usage: 1.3 GB


In [47]:
plants.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 388 entries, 0 to 387
Data columns (total 3 columns):
plant_type    388 non-null int8
plant         388 non-null int16
city          388 non-null int8
dtypes: int16(1), int8(2)
memory usage: 4.5 KB


# 3. Сохранение данных

Сохраним финальные данные в формате pickle для последующего использования. 

In [48]:
path = str(workdir+'/data')
if not os.path.isdir(path):
    os.mkdir(path)
    print('Папка успешно создана!')
else:
    print('Папка уже существует')

Папка уже существует


In [49]:
transactions.to_pickle(path+'/transactions.pkl')
clients.to_pickle(path+'/clients.pkl')
materials.to_pickle(path+'/materials.pkl')
plants.to_pickle(path+'/plants.pkl')

Сведения о соответствии изначальных идентификаторов категорий и с их целочисленным форматом не требуются. В рамках данной задачи нет необходимости в возврате к первоначальным идентификаторам. Однако, на случай необходимости оставим в данном блокноте таблицу с идентификаторами городов.

In [50]:
city_ids

,city,num
0,Other,1
3,Moscow,0
24,St. Petersburg,2
